In [1]:
##Python venv: search_agent_poc

In [1]:
##This module will take the serialized dictionary got out of PDF Parsing ann try to extract
##Semantic Knowldge like identifying 
## 1.Important Objects/Entities
## 2.Deduplicate Entities
## 3.Extracting Relations
## 4.Extract the main Ideas/Topics around Each Page
## 5.Link the different topics via diffrent entities/Objects
## 6.Break down the document by pages instead of Chunks .
## 7.If a page does not fit a chunk then chunk them extract information and then deduplicate the information across
## the page.

#Next Steps:
## 5.Try to Seggregate the BigPDF on Sections.
## 8.Try To Find Common Objects or ideas that link these sections.
## 9.Try 

In [34]:
import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser

In [35]:
model_name="llama3.1"
book_text=''
page_text=''
file_name='Kubernetes_in_action_text_only'
extension='.txt'
start_page_idx=306
end_page_index=479
pdf_parsed_output_dir = '../pdf_parsing/parsed_pdf_output_pickle/'
pdf_enrichment_output_dir = './pdf_enriched_output/'
#full_filename=file_name+'_'+str(page_idx)+extension
#full_filename

In [36]:
#LLM Model for Prompt Tuning
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name,temperature=0.2)

#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
model_kwargs = {'device': 'cpu'}

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs = model_kwargs)

##Define Vectorstore
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_kubernetes_in_action")

In [37]:
#Ininitial Load From Parsed Content (deserialized data)
#with open(pdf_parsed_output_dir+'pdf_content_dict_stage1.pickle', 'rb') as handle:
#    document_dict_deserialized = pickle.load(handle)

###Intermediate Load in case of errors
with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase1_entity_extraction_305.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [38]:
len(document_dict_deserialized)

479

In [39]:
#document_dict_deserialized=document_dict_deserialized[87:566]

In [40]:
len(document_dict_deserialized)

479

In [41]:
##Definitions of Individual Enrichment Modules######

# Function to reverse a string
def reverse(string):
    string = string[::-1]
    return string

def extract_entities_per_page(page_text):
    
    
    parser = XMLOutputParser()

# Prompt
    prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker\n
            Machine Larning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study a document page related to a particular technology and extract \n
            the main entities or objects from the document.The entities or objects can be anything surrounding \n
            technology in which your expetise lies.By Objects or entities i mean it can be a programming Language,\n
            command,framework,database,process,cache,controller,qeueues,schedulers,errors,exceptions or any \n
            components related to technology. \n
            surrounding the technology.Please output in form of list of json where each extracted entity will be \n
            represented by a json along with description and category.\n
            The category name can be like \n
            hardware,software,network,application,database,process,thread,container etc.Mention the category name \n
            in third bracket. \n
            After extracting all the entities along with their description and category format as per rules below: \n
            1.For each entity create a json with 3 keys "entity","description" and "category". \n
            2.Each extracted entity must be enlosed within double quotes. \n
            After all the entities of the page are extracted where each entity is a json then only collate all the entity json \m
            into a list of json.
            Output the collated list of json where each entity is represented by json. \n
            Do not output anything other than the list of json neither heading/decsription before the list of json\n
            nor any decsription/footer below the json \n
            Here is the document page: \n\n {context} \n\n""",
            input_variables=["context"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Chain
    #Commented out By Kaustav 20/08/2024 as str output parser is giving some unwanted text #####
    chain = prompt | llm | StrOutputParser()
   
    




    #Trying out XML Output parser by Kautsva added 20/08/2024####
    #chain = prompt | llm | parser
    
    

    # Score
    #filtered_docs = []
    
    #print("Page Text")
    #print(page_text)

    output = chain.invoke(
        {
                "context": page_text,
        }
    )
    
    #print('Relationship Output from LLM"')
    #print(output)
    
    if '[' in output:
        json_output=output.split('[')[1]
        json_output='['+json_output
    else:    
        json_output='['+json_output
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    

    json_output=reverse(json_output)
    
    if ']' in json_output:
        json_output=json_output.split(']')[1]
        json_output=reverse(json_output)
        json_output= json_output + ']'
    else:
        json_output=reverse(json_output)
        json_output= json_output + ']'
    
    return json_output
    
    #return page_output_json
    #return output

def entity_collector_per_page(entity_lst):

    entities=[]
    
    for entity in entity_lst:
        print("Entity:")
        print(entity)
        entity_name=entity['entity']
        entities.append(entity_name)
    return list(set(entities))


In [42]:
######This code enrcihes the PDF page by page and while doing so it fails after processing 20 to 50 pages####
######currently we are manually tackling this problem by making the pdf dict list into aglobal variable######
######After the llm fails we take the index of the last page done and save the pdf dict list uptil that ####
#####Then as of now i am manually restarting the by loading the saved pdf dict list as the source pdf and ###
#####Using the last done pdf page index +1 as the starting index.So (last Done page Index + 1) is the ######
###starting index###
####The maximum page has been given as 565 as that is the last page where the main content of the document###
####resides.In case of the book Kubernetes in action it is 565.##############################################
####If this is batch process then the last page can be manually seen and used as the variable else one can ####
#####also go for the pdf enrichment of all the pages of the book#############################################

###This manual thing needs to be automated by either langraph or by using agents framework##################

def enrich_page(page_idx):
    
    print("Page Number")
    print(page_idx)
    
    page_text=document_dict_deserialized[page_idx]['text']
    
    ##Entity Extraction Enrichment
    page_entity_lst_dict=extract_entities_per_page(page_text)
    
    #print("Page Entity List Dictionary")
    #print(page_entity_lst_dict)
    
    print("Page Entity List Dictionary Cleaned")
    print(page_entity_lst_dict)
    
    page_entity_lst_json=json.loads(page_entity_lst_dict)
    document_dict_deserialized[page_idx]['entities']=page_entity_lst_json
    
    ##Collate entity list per page into unique Entities
    
    entity_lst=entity_collector_per_page(page_entity_lst_json)
    
    ##Relationship Extraction Enrichment
    #page_relationship_lst_dict=extract_relationship_per_page(page_text,entity_lst)
    
    #print("Page Relationship List Dictionary")
    #print(page_entity_lst_dict)
    
    #document_dict_deserialized[page_idx]['relationships']=page_relationship_lst_dict
    print("Done for page "+str(page_idx))
    return page_idx

In [43]:

for idx in range(start_page_idx,end_page_index):
   
        enrich_page(idx)
        
        
        
        
    

Page Number
306
Page Entity List Dictionary Cleaned
[
  {
    "entity": "role-based access control",
    "description": "a security feature that allows administrators to define roles and permissions for users or groups within a cluster",
    "category": "software"
  },
  {
    "entity": "apiGroups",
    "description": "a field in Kubernetes role-based access control rules that specifies the API group(s) affected by the rule",
    "category": "software"
  },
  {
    "entity": "",
    "description": "the empty string, indicating that the rule applies to all API groups",
    "category": "software"
  },
  {
    "entity": "resources",
    "description": "a field in Kubernetes role-based access control rules that specifies the resources affected by the rule",
    "category": "software"
  },
  {
    "entity": "persistentvolumes",
    "description": "a resource type in Kubernetes that represents a persistent volume",
    "category": "software"
  },
  {
    "entity": "verbs",
    "description":

Page Entity List Dictionary Cleaned
[
  {
    "entity": "ClusterRole",
    "description": "A Kubernetes resource that defines a set of permissions for cluster-level access.",
    "category": "software"
  },
  {
    "entity": "ClusterRoleBinding",
    "description": "A Kubernetes resource that binds a ClusterRole to a user, service account, or group.",
    "category": "software"
  },
  {
    "entity": "RoleBinding",
    "description": "A Kubernetes resource that binds a Role to a user, service account, or group.",
    "category": "software"
  },
  {
    "entity": "/api",
    "description": "A non-resource URL exposed by the API server.",
    "category": "network"
  },
  {
    "entity": "/api/*",
    "description": "A non-resource URL pattern exposed by the API server.",
    "category": "network"
  },
  {
    "entity": "/apis",
    "description": "A non-resource URL exposed by the API server.",
    "category": "network"
  },
  {
    "entity": "/apis/*",
    "description": "A non-resource

Page Entity List Dictionary Cleaned
[
  {
    "entity": "ClusterRoleBinding",
    "description": "A binding that grants access to resources across all namespaces.",
    "category": "Kubernetes"
  },
  {
    "entity": "RoleBinding",
    "description": "A binding that grants access to resources within a specific namespace.",
    "category": "Kubernetes"
  },
  {
    "entity": "ClusterRole",
    "description": "A role that grants access to resources across all namespaces.",
    "category": "Kubernetes"
  },
  {
    "entity": "Role",
    "description": "A role that grants access to resources within a specific namespace.",
    "category": "Kubernetes"
  },
  {
    "entity": "ServiceAccount",
    "description": "An account used by pods to authenticate with the API server.",
    "category": "Kubernetes"
  },
  {
    "entity": "PodList",
    "description": "A list of pods in a specific namespace or across all namespaces.",
    "category": "Kubernetes"
  },
  {
    "entity": "curl",
    "descri

Page Entity List Dictionary Cleaned
[
  {"entity": "role-based access control", "description": "a security feature that allows you to manage access to a Kubernetes cluster based on roles", "category": "software"},
  {"entity": "ClusterRoleBindings", "description": "an object that binds a ClusterRole to a user or group", "category": "software"},
  {"entity": "ClusterRoles", "description": "a set of permissions that define what actions can be performed in a Kubernetes cluster", "category": "software"},
  {"entity": "kubectl", "description": "the command-line tool for interacting with a Kubernetes cluster", "category": "command"},
  {"entity": "get", "description": "a command used to retrieve information about an object in the cluster", "category": "command"},
  {"entity": "clusterrolebindings", "description": "an API resource that lists all ClusterRoleBindings in the cluster", "category": "software"},
  {"entity": "clusterroles", "description": "an API resource that lists all ClusterRole

Page Entity List Dictionary Cleaned
[
  {
    "entity": "Controller Manager",
    "description": "A single pod that runs each controller inside it",
    "category": "application"
  },
  {
    "entity": "ClusterRole",
    "description": "A role that defines permissions for a user or service account",
    "category": "database"
  },
  {
    "entity": "ClusterRoleBinding",
    "description": "A binding that associates a ClusterRole with a user or service account",
    "category": "database"
  },
  {
    "entity": "ServiceAccount",
    "description": "An identity for pods to authenticate with the API server",
    "category": "application"
  },
  {
    "entity": "Role",
    "description": "A role that defines permissions for a user or service account within a namespace",
    "category": "database"
  },
  {
    "entity": "RoleBinding",
    "description": "A binding that associates a Role with a user or service account within a namespace",
    "category": "database"
  },
  {
    "entity": "Cl

...

Page Entity List Dictionary Cleaned
[{"entity": "Kubernetes", "description": "Container orchestration system", "category": "software"}, 
 {"entity": "kubectl", "description": "Command-line tool for interacting with Kubernetes clusters", "category": "command"}, 
 {"entity": "Service Catalog", "description": "API for discovering and using services in a cluster", "category": "application"}, 
 {"entity": "ClusterServiceClass", "description": "Definition of a service that can be provisioned in a cluster", "category": "database"}, 
 {"entity": "StorageClasses", "description": "Definitions of available storage options for pods", "category": "database"}, 
 {"entity": "pods", "description": "Lightweight and portable containers", "category": "container"}, 
 {"entity": "apiVersion", "description": "Version of the API used to define a ClusterServiceClass", "category": "key-value pair"}, 
 {"entity": "bindable", "description": "Flag indicating whether a service can be bound to a cluster", "category

Page Entity List Dictionary Cleaned
[
  {"entity": "kubectl", "description": "command-line tool for managing Kubernetes clusters", "category": "software"},
  {"entity": "get secret", "description": "command to retrieve a Secret resource from a Kubernetes cluster", "category": "software"},
  {"entity": "postgres-secret", "description": "Secret resource holding database credentials", "category": "database"},
  {"entity": "apiVersion", "description": "API version of the Secret resource", "category": "software"},
  {"entity": "v1", "description": "API version of the Secret resource", "category": "software"},
  {"entity": "data", "description": "field containing base64-encoded database credentials", "category": "database"},
  {"entity": "host", "description": "field containing hostname of the database service", "category": "database"},
  {"entity": "username", "description": "field containing username for database connection", "category": "database"},
  {"entity": "password", "description":

Page Entity List Dictionary Cleaned
[
  {
    "entity": "Templates",
    "description": "A parameterizable list in OpenShift that allows a manifest to be instantiated with parameter values.",
    "category": "software"
  },
  {
    "entity": "BuildConfigs",
    "description": "A configuration for building images in OpenShift.",
    "category": "software"
  },
  {
    "entity": "DeploymentConfigs",
    "description": "A configuration for deploying applications in OpenShift.",
    "category": "software"
  },
  {
    "entity": "ImageStreams",
    "description": "A stream of images in OpenShift that can be used to deploy applications.",
    "category": "software"
  },
  {
    "entity": "Routes",
    "description": "A way to expose an application running in OpenShift to the outside world.",
    "category": "software"
  },
  {
    "entity": "ServiceAccounts",
    "description": "An object in Kubernetes that represents a service running in a cluster.",
    "category": "software"
  },
  {
    

Page Entity List Dictionary Cleaned
[{"entity": "Kubernetes", "description": "Container orchestration system", "category": "software"}, 
 {"entity": "DeploymentConfig", "description": "OpenShift resource for deploying applications", "category": "software"}, 
 {"entity": "ReplicationController", "description": "OpenShift resource for managing replicas of a pod", "category": "software"}, 
 {"entity": "Pod", "description": "Lightweight and portable container runtime", "category": "container"}, 
 {"entity": "Builder pod", "description": "OpenShift component for building images", "category": "component"}, 
 {"entity": "BuildConfig", "description": "OpenShift resource for configuring builds", "category": "software"}, 
 {"entity": "DeploymentConfig", "description": "OpenShift resource for deploying applications", "category": "software"}, 
 {"entity": "ImageStream", "description": "OpenShift resource for managing images", "category": "software"}, 
 {"entity": "Build trigger", "description": "O

Page Entity List Dictionary Cleaned
[
  {"entity": "Kubernetes", "description": "Container orchestration system", "category": "software"},
  {"entity": "Custom-ResourceDefinition", "description": "API server object for registering custom resources", "category": "software"},
  {"entity": "API server", "description": "Component of Kubernetes that manages API requests", "category": "software"},
  {"entity": "Controller", "description": "Kubernetes component that watches and responds to changes in the cluster", "category": "software"},
  {"entity": "Pods", "description": "Lightweight and portable container runtime environment", "category": "container"},
  {"entity": "Cluster", "description": "Group of machines that run Kubernetes", "category": "hardware/network"},
  {"entity": "Helm", "description": "Package manager for deploying applications to Kubernetes", "category": "software"},
  {"entity": "Resource manifests", "description": "Files that define the resources needed by an application"

In [10]:
document_dict_deserialized[467]

{'page': 556,
 'img_cnt': 0,
 'img_flag': 0,
 'img_npy_lst': [],
 'text': '524\nCHAPTER 18\nExtending Kubernetes\n18.2.4 Provisioning and using a service\nLet’s imagine the pods you’re deploying need to use a database. You’ve inspected the\nlist of available ClusterServiceClasses and have chosen to use the free plan of the\npostgres-database ClusterServiceClass. \nPROVISIONING A SERVICEINSTANCE\nTo have the database provisioned for you, all you need to do is create a Service-\nInstance resource, as shown in the following listing.\napiVersion: servicecatalog.k8s.io/v1alpha1\nkind: ServiceInstance\nmetadata:\n  name: my-postgres-db                     \nspec:\n  clusterServiceClassName: postgres-database        \n  clusterServicePlanName: free                             \n  parameters:\n    init-db-args: --data-checksums         \nYou created a ServiceInstance called my-postgres-db (that will be the name of the\nresource you’re deploying) and specified the ClusterServiceClass and the ch

In [45]:
import pickle

# Store data (serialize in a pickle) upto page 102
with open('./pdf_enriched_output/pdf_enriched_content_dict_phase1_entity_extraction_478_final.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
document_dict_deserialized[155]

{'page': 244,
 'img_cnt': 0,
 'img_flag': 0,
 'img_npy_lst': [],
 'text': '212\nCHAPTER 7\nConfigMaps and Secrets: configuring applications\nEDITING A CONFIGMAP\nLet’s see how you can change a ConfigMap and have the process running in the pod\nreload the files exposed in the configMap volume. You’ll modify the Nginx config file\nfrom your previous example and make Nginx use the new config without restarting\nthe pod. Try switching gzip compression off by editing the fortune-config Config-\nMap with kubectl edit:\n$ kubectl edit configmap fortune-config\nOnce your editor opens, change the gzip on line to gzip off, save the file, and then\nclose the editor. The ConfigMap is then updated, and soon afterward, the actual file\nin the volume is updated as well. You can confirm this by printing the contents of the\nfile with kubectl exec:\n$ kubectl exec fortune-configmap-volume -c web-server\n➥  cat /etc/nginx/conf.d/my-nginx-config.conf\nIf you don’t see the update yet, wait a while and try

In [ ]:
####The main contents of Kubernets In Action book is contained from page 87 to 565####
####Thats why we will only take the sliced portion i.e. from page 87 to 565 of the pdf dictionary list#####

In [48]:
# Load data (deserialize)
with open('pdf_content_dict_page_final.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [ ]:
# Store data (serialize in a pickle) upto page 102
with open('../pdf_enrichment/pdf_enriched_output/pdf_enriched_content_dict_stage1.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized, handle, protocol=pickle.HIGHEST_PROTOCOL)